**Molecular Dynamics: NVE**

If we want to work at conditions beyond the athermal limit, we need to do dynamical simulations. Here, we use a technique called molecular dynamics, or MD.

As previously, lets setup a surface and run some examples. First up, loading modules and building a Pt surface with Cu:

In [1]:
from ase import Atoms, Atom
from ase.build import fcc110
from math import sqrt

# Make the Pt 110 slab.
atoms = fcc110('Pt', (2, 2, 4), vacuum=7.)
atoms.set_pbc((1, 1, 0))

a = 3.9242
b = a / sqrt(2)
h = b / 2 
atoms.append(Atom('Cu',  atoms[8].position + (0., 0., 2.5)))

Now as usual lets assign the calcutor to the object:

In [2]:
from ase.calculators.emt import EMT
atoms.set_calculator(EMT())

OK, now we need to think about our MD simulation. Let's initially run the calculatio at 1000 K, using a Boltzmann distribution for the veolcities: 

In [3]:
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
# Set the momenta corresponding to T=500K
T = 1000
MaxwellBoltzmannDistribution(atoms, T*units.kB)

Next, lets setup an NVE calculation (remembering that NVE is constant mols, volume and energy):

In [4]:
from ase.md.verlet import VelocityVerlet
dyn = VelocityVerlet(atoms, 5 * units.fs)  # 5 fs time step.

Now we are going to run the calculation. We are going to use a helper function to print energies - don't worry about the code structure, this is just to help us see the results as the simulation progresses:

In [5]:
def printenergy(a=atoms):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

from ase.io.trajectory import Trajectory
# Now run the dynamics
dyn.attach(printenergy, interval=10)
traj = Trajectory('moldyn_nve.traj', 'w', atoms)
dyn.attach(traj.write, interval=10)
printenergy()

# Run for 200 steps, i.e. 200*5 fs = 1 ns
dyn.run(200)

Energy per atom: Epot = 0.327eV  Ekin = 0.129eV (T=1002K)  Etot = 0.457eV
Energy per atom: Epot = 0.327eV  Ekin = 0.129eV (T=1002K)  Etot = 0.457eV
Energy per atom: Epot = 0.379eV  Ekin = 0.078eV (T=603K)  Etot = 0.457eV
Energy per atom: Epot = 0.396eV  Ekin = 0.060eV (T=466K)  Etot = 0.457eV
Energy per atom: Epot = 0.374eV  Ekin = 0.083eV (T=639K)  Etot = 0.457eV
Energy per atom: Epot = 0.377eV  Ekin = 0.080eV (T=619K)  Etot = 0.457eV
Energy per atom: Epot = 0.374eV  Ekin = 0.083eV (T=640K)  Etot = 0.457eV
Energy per atom: Epot = 0.371eV  Ekin = 0.085eV (T=660K)  Etot = 0.457eV
Energy per atom: Epot = 0.376eV  Ekin = 0.080eV (T=621K)  Etot = 0.457eV
Energy per atom: Epot = 0.375eV  Ekin = 0.082eV (T=631K)  Etot = 0.457eV
Energy per atom: Epot = 0.378eV  Ekin = 0.078eV (T=606K)  Etot = 0.457eV
Energy per atom: Epot = 0.374eV  Ekin = 0.082eV (T=637K)  Etot = 0.457eV
Energy per atom: Epot = 0.359eV  Ekin = 0.097eV (T=753K)  Etot = 0.457eV
Energy per atom: Epot = 0.367eV  Ekin = 0.089eV (

True

Finally, once complete, lets visualise the system:

In [6]:
from ase.visualize import view
traj2 = Trajectory('moldyn_nve.traj')
view(traj2, viewer='ngl')

/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


/Users/andrewlogsdail/Library/Python/3.8/lib/python/site-packages/ase/io/jsonio.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  a = np.array(obj)


What is the behaviour of the system if you change the starting temperature, or the intial distribution of velocities? Alter the settings, rerun and see if your  data is as expected.

**Molecular Dynamics: NVT**

As our final calculation, lets run an NVT simulation and compare the results to our NVE.

We already have the settings and models loaded, so we can make use of this here and start our calculation from the end of the NVE simulation:

In [7]:
from ase.md.langevin import Langevin
dyn = Langevin(atoms, 5*units.fs, T*units.kB, 0.02)

All we need to do now the algorithm is setup is to run the calculation again:

In [8]:
# Now run the dynamics
dyn.attach(printenergy, interval=10)
traj = Trajectory('moldyn_nvt.traj', 'w', atoms)
dyn.attach(traj.write, interval=10)
printenergy()
dyn.run(200)

Energy per atom: Epot = 0.376eV  Ekin = 0.081eV (T=626K)  Etot = 0.457eV
Energy per atom: Epot = 0.376eV  Ekin = 0.081eV (T=626K)  Etot = 0.457eV
Energy per atom: Epot = 0.381eV  Ekin = 0.067eV (T=517K)  Etot = 0.447eV
Energy per atom: Epot = 0.373eV  Ekin = 0.069eV (T=535K)  Etot = 0.442eV
Energy per atom: Epot = 0.388eV  Ekin = 0.073eV (T=566K)  Etot = 0.461eV
Energy per atom: Epot = 0.396eV  Ekin = 0.086eV (T=662K)  Etot = 0.481eV
Energy per atom: Epot = 0.410eV  Ekin = 0.080eV (T=622K)  Etot = 0.490eV
Energy per atom: Epot = 0.395eV  Ekin = 0.092eV (T=715K)  Etot = 0.487eV
Energy per atom: Epot = 0.406eV  Ekin = 0.077eV (T=592K)  Etot = 0.483eV
Energy per atom: Epot = 0.398eV  Ekin = 0.102eV (T=786K)  Etot = 0.500eV
Energy per atom: Epot = 0.404eV  Ekin = 0.099eV (T=766K)  Etot = 0.503eV
Energy per atom: Epot = 0.385eV  Ekin = 0.103eV (T=800K)  Etot = 0.488eV
Energy per atom: Epot = 0.404eV  Ekin = 0.083eV (T=645K)  Etot = 0.487eV
Energy per atom: Epot = 0.388eV  Ekin = 0.103eV (T=

True

Notice how as the calculation runs the total energy decreases? Why is this?

Again, to visualise our results we can open the trajectory.

In [9]:
traj2 = Trajectory('moldyn_nvt.traj')
view(traj2, viewer='ngl')